# Workflow model evaluatie: Deel 3 Model vergelijking

Laatste aanpassing: 17 dec 2024 <br>
Contact: Petra Hulsman <br>

## Context

Binnen HDSR worden hydraulische modellen gebruikt voor het beantwoorden van verschillende vraagstukken. Bijvoorbeeld: Zitten er knelpunten in het systeem bij een T=1 zomer afvoersituatie? Hoeveel ruimte is er beschikbaar voor NVOs? Tot welke diepte moet minimaal gebaggerd worden? Welke dimensies moeten nieuwe kunstwerken bij voorkeur hebben om knelpunten te kunen voorkomen? Welke gebieden zijn kwetsbaar voor inundatie? Momenteel (anno begin 2024) wordt voornamelijk Sobek gebruikt om debieten en waterniveaus in primaire watergangen te modelleren. Het is van groot belang dat onze modellen betrouwbaar zijn.

In deze notebook is een workflow voor de model evaluatie opgezet. Deze bestaat uit drie delen:
1. Verificatie: model resultaten vs. beheerregister (BR)
2. Validatie: model resultaten vs. puntmetingen
3. Model vergelijking: model A vs. model B/satelliet data (n.b.: satelliet data zijn vaak ook het resultaat van modellen)

Deze workflow kan gebruikt worden om de betrouwbaarheid van onze modellen te kunnen bepalen en om model onnauwkeurigheden te kunnen identificeren en verbeteren. Deze evaluatie is sterk afhankelijk van de betrouwbaarheid van geobserveerde data, namelijk veld observaties en satelliet data die hiervoor gebruikt worden. Het is daarom erg belangrijk om te onthouden dat er niet alleen onzekerheden zijn in het model maar ook in de geobserveerde data. Daarom wordt aanbevolen kritisch naar de resultaten te kijken voordat een oordeel getroffen wordt.

## Uitvoer

Deze code werkt nu met de virtuale omgeving "Petra_arcpy" op de rekenserver reken05. Momenteel is een handmatige na-controle aan te raden om te verifieren of alle ruimtelijke koppelingen tussen model en data goed is gegaan.

Belangrijke informatie:
De FEWS-WIS data is m.b.v. een aparte script iets bewerkt voor dat het hier gebruikt wordt (vandaar ook de subscript "edited" in de bestandsnamen). Hierbij is op locaties waar er meerere pompen zijn, het totaal debiet toegevoegd.

### Afvoer (laterals): RUPROF vs. Hydromedah
<br>
<u>Variabele</u>:  <br>
- Afvoer (i.e. lateral) voor T=1 in de zomer  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) hydrologisch/grondwater model (HYDROMEDAH), en 2) karakterestieke waarden per landgebruik (RUPROF input) <br>
<u>Output</u>: <br>
- Shapefile (polygons): verschil tussen HYDROMEDAH en RUPROF <br>
- Figuren: verschil per afwateringsgebied (gebieden met dQ = 0.0 m zijn grijs gemarkeerd)

In [ ]:
import os
import numpy as np
import functies_punt_laterals_T1

class paths():
    # Input bestanden
    root          = os.getcwd()
    shp_RchSeg    = os.path.join(root, 'GIS input','RchSegments.shp') # shapefile van de reach segments, export uit Sobek
    shp_node      = os.path.join(root, 'GIS input','Sbk_Lat_n.shp') # shapefile van de lateralen punten, export uit Sobek
    dat_Sobek     = os.path.join(root, 'Input','Sobek output','MOD_ADJ_V4.lit') # locatie Sobek HIS output bestanden
    dat_lat       = os.path.join(root, 'Input','Sobek output','LATERAL.DAT') # lateralen data, Sobek input data
    shp_afvoer    = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    shp_afwatr    = os.path.join(root,"GIS input","Afwateringseenheden_2020.shp") # shapefile van de afwateringsgebieden
    
     # Output bestanden
    shp_laterals  = os.path.join(root,"GIS output",'laterals_T1_RUPROF_HYDROMEDAH.shp') # shapefile met lateralen data op basis van HYDROMEDAH, RUPROF en het verschil
    shp_HYDROMEDAH= os.path.join(root,"GIS output",'laterals_T1_HYDROMEDAH.shp') # shapefile met lateralen data op basis van HYDROMEDAH, alleen opgeslagen als er iets mis gaat bij het converteren van reach segments naar lateralen knopen (u wordt geinformeerd als dit gebeurt)
    shp_RUPROF    = os.path.join(root,"GIS output",'laterals_T1_RUPROF.shp') # shapefile met lateralen data op basis van RUPROF
    fightml       = os.path.join(root,"Output",'3_fig_laterals_T1.html') # Figuur van de verschillen in lateralen data

months = [5,6,7,8,9] # maanden in het jaar om statistiek over te berekenen
years  = np.arange(2010,2022) # zie Sobek continue model time-range
functies_punt_laterals_T1.main(paths,months,years)

Note: Handmatige controle voor de ruimtelijke koppeling van HYDROMEDAH debieten (RchSegments) naar lateral nodes (Sbk_Lat_n). De afstand is groot op 2 locaties, maar daar is deze koppeling ook goed gegaan. Om een of andere reden is daar geen rStor link.

### Afvoer (laterals): Hydromedah vs. satelliet
<br>
<u>Variabele</u>:  <br>
- Gemiddelde afvoer (i.e. lateral) richting primaire watergang  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) satelliet data (FEWS_WIS), en 2) model (HYDROMEDAH, via laterals in Sobek model) <br>
<u>Randvoorwaarden</u>: <br>
- Gemiddelde afvoer per peilgebied <br>
<u>Output</u>: <br>
- Shapefile (polygons): verschil tussen data en model op afvoergebiedniveau<br>
- Figuren: verschil per afwateringsgebied

In [ ]:
import os
import numpy as np
import functies_ruimtelijk_afvoer_gem

class paths():
    # Input bestanden
    root          = os.getcwd()
    shp_RchSeg    = os.path.join(root, 'GIS input','RchSegments.shp') # shapefile van de reach segments, export uit Sobek
    shp_node      = os.path.join(root, 'GIS input','Sbk_Lat_n.shp') # shapefile van de lateralen punten, export uit Sobek
    dat_Sobek     = os.path.join(root, 'Input','Sobek output','MOD_ADJ_V4.lit') # locatie Sobek HIS output bestanden
    dat_lat       = os.path.join(root, 'Input','Sobek output','LATERAL.DAT') # lateralen data, Sobek input data
    shp_afvoer    = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    shp_afwatr    = os.path.join(root,"GIS input","Afwateringseenheden_2020.shp") # shapefile van de afwateringsgebieden
    shp_peil      = os.path.join(root,"GIS input","BR-VS_Peilgebieden_2010_FEWSWIS.shp") # shapefile van de peilgebieden zoals gebruikt in FEWS-WIS
    df_P          = os.path.join(root,'Input','FEWS-WIS','P_KNMI_2000_2024_peilgebieden.csv') # jaarlijkse regen data
    df_E          = os.path.join(root,'Input','FEWS-WIS','E_ELEAF_2000_2024_peilgebieden.csv') # jaarlijkse verdamping data
    raster_kwelO  = os.path.join(root,'GIS input','Oost_HUI_BDGFLF_MEAN_2012-1-1_TO_2019-12-31_L2_MMPD.tif') # kwel raster
    raster_kwelW  = os.path.join(root,'GIS input','Veenweide_BDGFLF_MEAN_2010-1-1_TO_2020-12-31_L2_MMD.tif') # kwel raster
        
    # Output bestanden
    shp_kwel       = os.path.join(root,"GIS output",'kwel.shp') # shapefile met afvoer data op basis van HYDROMEDAH, satelliet data en het verschil
    shp_comp       = os.path.join(root,"GIS output",'laterals_RS_gem_HYDROMEDAH_data.shp') # shapefile met afvoer data op basis van HYDROMEDAH, satelliet data en het verschil
    shp_sat        = os.path.join(root,"GIS output",'laterals_RS_gem_SATELLIET.shp') # shapefile met afvoer data op basis van satelliet data
    shp_HYDROMEDAH = os.path.join(root,"GIS output",'laterals_RS_gem_HYDROMEDAH.shp') # shapefile met afvoer data op basis van HYDROMEDAH
    fightml       = os.path.join(root,"Output",'3_fig_laterals_sat_gem.html') # Figuur van de verschillen in afvoer
    

months = [5,6,7,8,9] # maanden in het jaar om statistiek over te berekenen
years  = np.arange(2010,2022) # zie Sobek continue model time-range
functies_ruimtelijk_afvoer_gem.main(paths,months,years)
